### Import Libraries

In [1]:
import os
import torch
import numpy as np
import torchio as tio
import SimpleITK as sitk

from pathlib import Path
from tqdm.notebook import tqdm
from torchio.transforms import (Compose,
                                ToCanonical,
                                RescaleIntensity, 
                                CropOrPad)

### Download Raw Data

In [ ]:
DATA_PATH = '../data/raw/'

'''Transformation'''
ixi_dataset = tio.datasets.IXI(DATA_PATH,
                               modalities=('T1','T2','PD',),
                               download=True)

### Process Data

In [2]:
"""Load Data"""
directory = '../data/raw/masks/'

Path('../data/processed_IXI/').mkdir(parents=True, exist_ok=True)
Path('../data/processed_IXI/T1_mask').mkdir(parents=True, exist_ok=True)
Path('../data/processed_IXI/T2_mask').mkdir(parents=True, exist_ok=True)
Path('../data/processed_IXI/PD_mask').mkdir(parents=True, exist_ok=True)

t2_paths = []
subjects_name =[]
for d in np.sort(os.listdir(directory+'T2_mask/')):
    if 'ipynb' in d:
        continue
    if 'IXI580-IOP-1157' not in directory+'T2_mask/'+d:
        subjects_name += [d[:-7]] 
        t2_paths += [directory+'T2_mask/'+d]
print(t2_paths)
t1_paths = []
for d in subjects_name:
    if not os.path.exists(directory+'T1_mask'+'/'):
        raise Exception('PATH does not exist')
    t1_paths += [directory+'T1_mask'+'/'+d+'.nii.gz']
print(t1_paths)
pd_paths = []
for d in subjects_name: 
    if not os.path.exists(directory+'PD_mask'+'/'):
        raise Exception('PATH does not exist')    
    pd_paths += [directory+'PD_mask'+'/'+d+'.nii.gz']

print(pd_paths)
'''Load'''
_mod = ['T2_mask','T1_mask','PD_mask']
for j, paths in enumerate([t2_paths, t1_paths, pd_paths]):
    print('Processing {}'.format(_mod[j]))
    subjects = []
    for i in range(len(paths)):
        subject = tio.Subject(mri= tio.ScalarImage(paths[i]),
                              name=subjects_name[i])
        subjects.append(subject)

    '''Transformation'''
    intensity_norm = RescaleIntensity((0,1),
                                      (0.5, 99.5))

    transforms = Compose([tio.Resample(1),
                          intensity_norm,
                          ToCanonical(1), 
                          CropOrPad(target_shape=(192,192,256))])

    train_set = tio.SubjectsDataset(subjects, 
                                    transform=transforms)

    train_loader = torch.utils.data.DataLoader(train_set,
                                               batch_size=1,
                                               shuffle=False, 
                                               drop_last=False, 
                                               num_workers=0)
    
    for i,data in tqdm(enumerate(train_loader)):
        x = data['mri'][tio.DATA].squeeze().numpy()
        name = data['name'][0]
        x = sitk.GetImageFromArray(x)
        sitk.WriteImage(x, '../data/processed_IXI/{}/{}_mask.nii.gz'.format(_mod[j], name))

['../data/raw/masks/T2_mask/ThoraxCBCT_0000_0000.nii.gz', '../data/raw/masks/T2_mask/ThoraxCBCT_0000_0001.nii.gz', '../data/raw/masks/T2_mask/ThoraxCBCT_0000_0002.nii.gz', '../data/raw/masks/T2_mask/ThoraxCBCT_0001_0000.nii.gz', '../data/raw/masks/T2_mask/ThoraxCBCT_0001_0001.nii.gz', '../data/raw/masks/T2_mask/ThoraxCBCT_0001_0002.nii.gz', '../data/raw/masks/T2_mask/ThoraxCBCT_0002_0000.nii.gz', '../data/raw/masks/T2_mask/ThoraxCBCT_0002_0001.nii.gz', '../data/raw/masks/T2_mask/ThoraxCBCT_0002_0002.nii.gz', '../data/raw/masks/T2_mask/ThoraxCBCT_0003_0000.nii.gz', '../data/raw/masks/T2_mask/ThoraxCBCT_0003_0001.nii.gz', '../data/raw/masks/T2_mask/ThoraxCBCT_0003_0002.nii.gz', '../data/raw/masks/T2_mask/ThoraxCBCT_0004_0000.nii.gz', '../data/raw/masks/T2_mask/ThoraxCBCT_0004_0001.nii.gz', '../data/raw/masks/T2_mask/ThoraxCBCT_0004_0002.nii.gz', '../data/raw/masks/T2_mask/ThoraxCBCT_0005_0000.nii.gz', '../data/raw/masks/T2_mask/ThoraxCBCT_0005_0001.nii.gz', '../data/raw/masks/T2_mask/Tho

0it [00:00, ?it/s]

c:\AMS_IZIV\AMS_IZIV_keymorph\.venv\lib\site-packages\torchio\data\image.py:248: UserWarning: Using TorchIO images without a torchio.SubjectsLoader in PyTorch >= 2.3 might have unexpected consequences, e.g., the collated batches will be instances of torchio.Subject with 5D images. Replace your PyTorch DataLoader with a torchio.SubjectsLoader so that the collated batch becomes a dictionary, as expected. See https://github.com/fepegar/torchio/issues/1179 for more context about this issue.
  warnings.warn(message, stacklevel=1)


Processing T1_mask


0it [00:00, ?it/s]

Processing PD_mask


0it [00:00, ?it/s]